## Description:

This notebook contains the implementation of a simple multiple agent system using the `llama_index` framework. The implementation includes three agents:
1. **DefaultResponseAgent**: Handles general questions (like greetings, general inquires, ...).
2. **BasicMathAgent**: Performs basic mathematical operations.
3. **RouterAgent**: Root Agent. Its main task is to route tasks to the appropriate agent based on the input.

This notebook demonstrates that an agent (here `BasicMathAgent`) can utilize tools when it is initialized as a single agent within the `AgentWorkflow`. However, when the same agent is included in the multi-agent system, it is unable to use the tools and provides random responses instead.

In [1]:
from llama_index.core.agent.workflow import (
    AgentWorkflow, 
    AgentInput,
    AgentOutput,
    AgentStream,
    FunctionAgent,
    ToolCall,
    ToolCallResult,
)
from llama_index.core.settings import Settings
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama

In [2]:
Settings.llm = Ollama(
    base_url="http://localhost:11434",
    model="llama3.1:8b",
    temperature= 0,
)

## Implement `basic_math_agent`

In [3]:

BASIC_MATH_SYSTEM_PROMPT = """
You are BasicMathAgent, an assistant specialized in performing basic mathematical operations. \
Your primary responsibility is to help users perform addition and subtraction calculations accurately.

You must use the `add_two_numbers` tool when users ask to add numbers together, \
and the `subtract_two_numbers` tool when users ask to subtract numbers. \
You cannot perform these calculations on your own or provide mathematical results without using the appropriate tools.

When users ask questions involving addition or subtraction, always use the corresponding tool to provide accurate results.
After receiving information from the tool, format your response directly to the user with the relevant calculation result.
Never say phrases like "the calculation is complete" or refer to yourself using the tool.
Instead, provide a natural, direct response that clearly presents the calculation result.

If users request mathematical operations beyond addition and subtraction, \
clearly explain that you can only perform addition and subtraction at this time.

If users ask questions completely unrelated to basic math operations, \
you should transfer control to the `DefaultResponseAgent` by using the `handoff` tool.
"""


async def add_two_numbers(num1: int, num2: int) -> int:
    """Useful for adding two numbers together."""
    return num1 + num2

async def subtract_two_numbers(num1: int, num2: int) -> int:
    """Useful for subtracting two numbers."""
    return num1 - num2


add_tool = FunctionTool.from_defaults(
    async_fn=add_two_numbers,
    description= "Useful for adding two numbers.",
)

sub_tool = FunctionTool.from_defaults(
    async_fn=subtract_two_numbers,
    description= "Useful for subtracting two numbers.",
)

basic_math_agent = FunctionAgent(
    name="BasicMathAgent",
    description= "A Basic Math Agent that can perform addition and subtraction operations.",
    system_prompt= BASIC_MATH_SYSTEM_PROMPT,
    llm=Settings.llm,
    tools = [add_tool, sub_tool],
    can_handoff_to= ["DefaultResponseAgent"],
)

#### Sanity Check for BasicMathAgent

This section performs a sanity check to verify whether the `BasicMathAgent` can utilize the provided tools when it is initialized as a single agent within the `AgentWorkflow`. The purpose of this test is to ensure that the `BasicMathAgent` is correctly set up and can perform basic mathematical operations as expected.

In [4]:
single_agent_workflow = AgentWorkflow(
    agents=[basic_math_agent]
)

handler = single_agent_workflow.run(
    user_msg= "Hello, What is 2 + 2?",
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    
    if isinstance(event, AgentInput):
        print(f"Chat history passed onto {current_agent}: ")
        print("*****Chat History Start*****")
        for message in event.input:
            print(f"{message.role}: {message.content}")
        print("*****Chat History End*****")
        print()
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")
    # if isinstance(event, AgentStream):
    #     print(event.delta, end="", flush=True)



🤖 Agent: BasicMathAgent

Chat history passed onto BasicMathAgent: 
*****Chat History Start*****
system: 
You are BasicMathAgent, an assistant specialized in performing basic mathematical operations. Your primary responsibility is to help users perform addition and subtraction calculations accurately.

You must use the `add_two_numbers` tool when users ask to add numbers together, and the `subtract_two_numbers` tool when users ask to subtract numbers. You cannot perform these calculations on your own or provide mathematical results without using the appropriate tools.

When users ask questions involving addition or subtraction, always use the corresponding tool to provide accurate results.
After receiving information from the tool, format your response directly to the user with the relevant calculation result.
Never say phrases like "the calculation is complete" or refer to yourself using the tool.
Instead, provide a natural, direct response that clearly presents the calculation result

The `BasicMathAgent` has successfully passed the sanity check, demonstrating its ability to perform basic mathematical operations using the provided tools. This confirms that the agent is functioning as intended and is ready for integration into a more complex multi-agent system.

## Implement `default_response_agent` 

In [5]:
DEFAULT_RESPONSE_AGENT_DESCRIPTION = """\
Useful for handling general user inquiries, common greetings, casual conversations, and open-domain questions. \
Provides default responses and basic interaction support.
"""

DEFAULT_RESPONSE_AGENT_SYSTEM_PROMPT = """\
You are DefaultResponseAgent. \
Your primary foucs is to assist users with general inquiries, common greetings, \
casual conversations, and open-domain questions. 

Follow these key guidelines EVERY TIME:
- Always provide direct, concise, and truthful answers (1-2 sentences max). \
STRICTLY Do NOT include any information beyond what is explicitly asked.
- Never start your answers with 'As an AI language model' or 'From the documents \
provided'. Avoid writing a disclaimer.
- If unsure, state 'I don't have enough information to answer that.' \
Do NOT make up answers any answers if you are unsure.
"""

default_response_agent = FunctionAgent(
    name= "DefaultResponseAgent",
    description= DEFAULT_RESPONSE_AGENT_DESCRIPTION,
    system_prompt= DEFAULT_RESPONSE_AGENT_SYSTEM_PROMPT,
    tools = [],
    can_handoff_to= [],
    llm= Settings.llm
)

## Implement `router_agent`. This agent acts as the root agent in the multi agent system

In [6]:
ROUTER_AGENT_SYSTEM_PROMPT = """
You are a ROUTER AGENT. \
You are the single point of contact for all user interactions. \ 
Understand user requests, route them to the appropriate specialized agent \
using the tool `handoff`, and present the response to the user without modification.

Instructions:
1. Receive and analyze the user's query.
2. Determine if the query falls under the expertise of a specialized agent \
using available tools.
"""


ROUTER_AGENT_DESCRIPTION= """\
A router agent, that handles initial query processing, routes requests to appropriate agents"""

router_agent = FunctionAgent(
    name = "RouterAgent",
    description = ROUTER_AGENT_DESCRIPTION,
    system_prompt= ROUTER_AGENT_SYSTEM_PROMPT,
    tools = [],
    can_handoff_to= [default_response_agent.name, basic_math_agent.name],
)

## Intialize `AgentWorkflow` for multi-agent system

In [7]:
workflow = AgentWorkflow(
    agents= [router_agent, default_response_agent, basic_math_agent],
    initial_state= {},
    root_agent= router_agent.name,
)

In [9]:
handler = workflow.run(
    user_msg= "Hello, What is 2 + 2?",
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    
    if isinstance(event, AgentInput):
        print(f"Chat history passed onto {current_agent}: ")
        print("*****Chat History Start*****")
        for message in event.input:
            print(f"{message.role}: {message.content}")
        print("*****Chat History End*****")
        print()
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")
    # if isinstance(event, AgentStream):
    #     print(event.delta, end="", flush=True)



🤖 Agent: RouterAgent

Chat history passed onto RouterAgent: 
*****Chat History Start*****
system: 
You are a ROUTER AGENT. You are the single point of contact for all user interactions. \ 
Understand user requests, route them to the appropriate specialized agent using the tool `handoff`, and present the response to the user without modification.

Instructions:
1. Receive and analyze the user's query.
2. Determine if the query falls under the expertise of a specialized agent using available tools.

user: Hello, What is 2 + 2?
*****Chat History End*****

🛠️  Planning to use tools: ['handoff']
🔨 Calling Tool: handoff
  With arguments: {'reason': 'math operation', 'to_agent': 'BasicMathAgent'}
🔧 Tool Result (handoff):
  Arguments: {'reason': 'math operation', 'to_agent': 'BasicMathAgent'}
  Output: Agent BasicMathAgent is now handling the request due to the following reason: math operation.
Please continue with the current request.

🤖 Agent: BasicMathAgent

Chat history passed onto BasicM

The above print statements captures the events during the execution of the `AgentWorkflow` with multiple agents. It shows the interactions between the agents and the tools, highlighting the issue where the `BasicMathAgent` does not call any tools as expected, when included in a multi-agent framework. Despite being able to use tools when initialized as a single agent, the `BasicMathAgent` fails to do so in a multi-agent setup and provides random responses instead.

## Conclusion
So here's what I found - there's definitely something fishy going on with the tools in LlamaIndex. When I run the agent by itself in AgentWorkflow, it has no problem using the tools I gave it. But as soon as I put that same agent into a multi-agent setup, it completely ignores the tools and just makes up random responses instead.

I am not sure whether there is something wrong with my implementation or a bug in how LlamaIndex handles tool access when multiple agents are involved. The tools seem to get lost in translation somehow when moving from single-agent to multi-agent workflows.